In [24]:
from openbb import obb

def get_csp_options(symbol, annualized_return_threshold):
    # Get the option chains for the specified symbol
    chains = obb.derivatives.options.chains(symbol=symbol)
    df = chains.to_dataframe()

    # Filter for put options
    put_options = df[df['option_type'] == 'put']

    # Get the current underlying price
    current_price = df['underlying_price'].iloc[0]

    # Function to calculate annualized return
    def calculate_annualized_return(row):
        days_to_expiration = row['dte']
        strike_price = row['strike']
        bid_price = row['bid']
        
        if days_to_expiration == 0 or strike_price == 0:
            return 0
        
        # Calculate potential profit
        potential_profit = bid_price
        
        # Calculate annualized return
        annualized_return = (potential_profit / strike_price) * (365 / days_to_expiration) * 100
        
        return annualized_return

    # Apply the function to each row
    put_options['annualized_return'] = put_options.apply(calculate_annualized_return, axis=1)

    # Filter for options with annualized return higher than the threshold and strikes under the current price
    high_return_puts = put_options[(put_options['annualized_return'] >= annualized_return_threshold) & (put_options['strike'] < current_price)]

    # Sort by strike in descending order
    high_return_puts = high_return_puts.sort_values('dte', ascending=True)

    # Select relevant columns
    result_list = high_return_puts[['contract_symbol', 'strike', 'dte', 'bid', 'annualized_return']].to_dict('records')

    print(f"Current underlying price: ${current_price:.2f}")
    print(f"Put options with {annualized_return_threshold}% or higher annualized return if sold as cash-secured puts (strikes under current price):")
    for option in result_list:
        print(f"Contract: {option['contract_symbol']}, Strike: ${option['strike']:.2f}, DTE: {option['dte']}, Bid: ${option['bid']:.2f}, Annualized Return: {option['annualized_return']:.2f}%")

    print(f"\nTotal number of qualifying options: {len(result_list)}")
    return result_list



In [22]:
get_csp_options('PINS', 30)

Current underlying price: $32.34
Put options with 30% or higher annualized return if sold as cash-secured puts (strikes under current price):
Contract: PINS241004P00031500, Strike: $31.50, DTE: 4, Bid: $0.18, Annualized Return: 52.14%
Contract: PINS241004P00032000, Strike: $32.00, DTE: 4, Bid: $0.33, Annualized Return: 94.10%
Contract: PINS241011P00031500, Strike: $31.50, DTE: 11, Bid: $0.42, Annualized Return: 44.24%
Contract: PINS241011P00032000, Strike: $32.00, DTE: 11, Bid: $0.60, Annualized Return: 62.22%
Contract: PINS241018P00031500, Strike: $31.50, DTE: 18, Bid: $0.58, Annualized Return: 37.34%
Contract: PINS241018P00032000, Strike: $32.00, DTE: 18, Bid: $0.78, Annualized Return: 49.43%
Contract: PINS241025P00031000, Strike: $31.00, DTE: 25, Bid: $0.66, Annualized Return: 31.08%
Contract: PINS241025P00032000, Strike: $32.00, DTE: 25, Bid: $1.03, Annualized Return: 46.99%
Contract: PINS241101P00032000, Strike: $32.00, DTE: 32, Bid: $1.60, Annualized Return: 57.03%
Contract: PINS

d:\repos\Trading-Strategies\venv\lib\site-packages\openbb_yfinance\models\options_chains.py:140: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chain.fillna("N/A").replace("N/A", None).to_dict("records")
C:\Users\SamAl\AppData\Local\Temp\ipykernel_5036\4140479914.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  put_options['annualized_return'] = put_options.apply(calculate_annualized_return, axis=1)


[{'contract_symbol': 'PINS241004P00031500',
  'strike': 31.5,
  'dte': 4,
  'bid': 0.18,
  'annualized_return': 52.142857142857146},
 {'contract_symbol': 'PINS241004P00032000',
  'strike': 32.0,
  'dte': 4,
  'bid': 0.33,
  'annualized_return': 94.1015625},
 {'contract_symbol': 'PINS241011P00031500',
  'strike': 31.5,
  'dte': 11,
  'bid': 0.42,
  'annualized_return': 44.242424242424235},
 {'contract_symbol': 'PINS241011P00032000',
  'strike': 32.0,
  'dte': 11,
  'bid': 0.6,
  'annualized_return': 62.21590909090908},
 {'contract_symbol': 'PINS241018P00031500',
  'strike': 31.5,
  'dte': 18,
  'bid': 0.58,
  'annualized_return': 37.33686067019401},
 {'contract_symbol': 'PINS241018P00032000',
  'strike': 32.0,
  'dte': 18,
  'bid': 0.78,
  'annualized_return': 49.427083333333336},
 {'contract_symbol': 'PINS241025P00031000',
  'strike': 31.0,
  'dte': 25,
  'bid': 0.66,
  'annualized_return': 31.083870967741934},
 {'contract_symbol': 'PINS241025P00032000',
  'strike': 32.0,
  'dte': 25,


In [31]:
from datetime import datetime, timedelta

sma_period = 200
rsi_period = 14

# Calculate the start date
end_date = datetime.now()
start_date = (end_date - timedelta(days=max(sma_period, rsi_period * 7))).strftime('%Y-%m-%d')

# Fetch stock data for NVDL
stock_data = obb.equity.price.historical("NVDL", start_date=start_date, interval="1d")

stock_data_df = stock_data.to_dataframe()

# Calculate 200-day Simple Moving Average (SMA)
sma_200 = obb.technical.sma(stock_data_df['Close'], sma_period).iloc[-1]

# Calculate weekly RSI
weekly_data = stock_data.resample('W').last()
weekly_rsi = obb.technical.rsi(weekly_data['Close'], rsi_period).iloc[-1]

# Calculate weekly MACD
weekly_macd = obb.technical.macd(weekly_data['Close'])
macd_line = weekly_macd['MACD'].iloc[-1]
signal_line = weekly_macd['Signal'].iloc[-1]
macd_histogram = weekly_macd['Histogram'].iloc[-1]

print(f"200-day SMA: {sma_200:.2f}")
print(f"Weekly RSI: {weekly_rsi:.2f}")
print(f"Weekly MACD:")
print(f"  MACD Line: {macd_line:.2f}")
print(f"  Signal Line: {signal_line:.2f}")
print(f"  Histogram: {macd_histogram:.2f}")


AttributeError: 'App' object has no attribute 'technical'